# PyOutreg Tutorial: Complete Guide to Regression Export in Python

## 🎯 Overview

**PyOutreg** is a Python implementation of Stata's popular `outreg2` command, designed to export regression results and statistical tables to Excel and Word formats with publication-quality formatting.

### Key Features:
- ✅ **Regression Export**: OLS, Logit, Probit, Panel models
- ✅ **Multiple Model Comparison**: Side-by-side tables
- ✅ **Summary Statistics**: Descriptive stats with customization
- ✅ **Cross-tabulation**: Categorical variable analysis
- ✅ **Professional Formatting**: Excel (.xlsx) and Word (.docx) export
- ✅ **Extensive Customization**: Titles, notes, decimal places, variable selection

### What You'll Learn:
1. Basic regression export to Excel/Word
2. Comparing multiple models in single tables
3. Creating summary statistics and cross-tabulations
4. Advanced customization options
5. Panel data models (fixed/random effects)
6. Logistic regression with odds ratios

Let's get started! 🚀

## 1. Import Required Libraries and Load Sample Data

We'll start by importing all necessary libraries and creating sample data for our demonstrations.

In [14]:
# Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Regression libraries
import statsmodels.api as sm
import linearmodels.panel as lmp

# PyOutreg - our star of the show!
import pyoutreg
from pyoutreg import outreg, summary_stats, cross_tab, outreg_compare

print("✅ All libraries imported successfully!")
print("📦 PyOutreg version:", pyoutreg.__version__ if hasattr(pyoutreg, '__version__') else "Development")

✅ All libraries imported successfully!
📦 PyOutreg version: 0.1.0


In [15]:
# Create sample dataset for our tutorial
np.random.seed(42)  # For reproducible results
n = 1000

# Generate sample data
data = pd.DataFrame({
    # Continuous variables
    'income': np.random.normal(50000, 15000, n),
    'education': np.random.normal(16, 3, n),
    'experience': np.random.normal(10, 8, n),
    'age': np.random.normal(35, 10, n),
    
    # Categorical variables  
    'gender': np.random.choice(['Male', 'Female'], n),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n),
    'employment': np.random.choice(['Full-time', 'Part-time', 'Unemployed'], n),
    
    # Panel data identifiers
    'id': np.repeat(range(100), 10),  # 100 individuals
    'year': np.tile(range(2014, 2024), 100),  # 10 years each
})

# Create a binary dependent variable for logistic regression
data['employed'] = (data['employment'] != 'Unemployed').astype(int)

# Create a realistic dependent variable with some relationship
data['wage'] = (
    20000 + 
    500 * data['education'] + 
    300 * data['experience'] + 
    200 * data['age'] + 
    5000 * (data['gender'] == 'Male') +
    np.random.normal(0, 5000, n)
)

# Clean up the data
data['income'] = np.clip(data['income'], 20000, 120000)
data['education'] = np.clip(data['education'], 8, 25)
data['experience'] = np.clip(data['experience'], 0, 40)
data['age'] = np.clip(data['age'], 18, 65)
data['wage'] = np.clip(data['wage'], 15000, 150000)

print(f"📊 Sample data created:")
print(f"   • {len(data):,} observations")
print(f"   • {len(data.columns)} variables")
print(f"   • Variables: {', '.join(data.columns)}")
print("\n🔍 First few rows:")
data.head()

📊 Sample data created:
   • 1,000 observations
   • 11 variables
   • Variables: income, education, experience, age, gender, region, employment, id, year, employed, wage

🔍 First few rows:


,income,education,experience,age,gender,region,employment,id,year,employed,wage
0,57450.712295,20.198066,4.598574,18.000000,Female,East,Part-time,0,2014,1,35538.864926
1,47926.035482,18.773901,8.843851,26.396150,Male,East,Full-time,0,2015,1,37492.953995
2,59715.328072,16.178891,3.660641,30.863945,Female,South,Full-time,0,2016,1,35002.509320
3,72845.447846,14.059190,7.536308,53.876877,Female,East,Part-time,0,2017,1,45723.754735
4,46487.699379,18.094670,0.000000,40.565531,Female,North,Unemployed,0,2018,0,27568.161103


## 2. Basic Regression Export

Let's start with the most common use case: fitting a simple OLS regression and exporting the results to Excel and Word formats.

### 📋 What we'll cover:
- Preparing data for regression
- Fitting an OLS model with statsmodels
- Exporting to Excel (.xlsx)
- Exporting to Word (.docx)
- Understanding the output format

In [3]:
# Step 1: Prepare variables for regression
y = data['wage']  # Dependent variable
X = data[['education', 'experience', 'age']]  # Independent variables
X = sm.add_constant(X)  # Add intercept term

# Step 2: Fit the OLS regression
model = sm.OLS(y, X)
result = model.fit()

# Step 3: Display basic results
print("🎯 OLS Regression Results Summary:")
print(f"   • R-squared: {result.rsquared:.3f}")
print(f"   • Adjusted R-squared: {result.rsquared_adj:.3f}")
print(f"   • Number of observations: {int(result.nobs):,}")
print(f"   • F-statistic: {result.fvalue:.2f}")
print(f"   • F-statistic p-value: {result.f_pvalue:.4f}")

print("\n📊 Coefficient Summary:")
print(result.params.round(2))

🎯 OLS Regression Results Summary:
   • R-squared: 0.260
   • Adjusted R-squared: 0.258
   • Number of observations: 1,000
   • F-statistic: 116.88
   • F-statistic p-value: 0.0000

📊 Coefficient Summary:
const         20441.82
education       597.98
experience      326.47
age             192.51
dtype: float64


In [4]:
# Export to Excel format
outreg(result, 'tutorial_basic_regression.xlsx', 
       title="Basic OLS Regression Results",
       ctitle="Model 1",
       replace=True)

print("✅ Results exported to 'tutorial_basic_regression.xlsx'")
print("📁 Check your working directory for the Excel file!")

# Let's also preview what the table looks like
print("\n🔍 Preview of the formatted table:")
preview_table = outreg(result, filename=None, ctitle="Model 1")
print(preview_table.to_string(index=False))

Results exported to tutorial_basic_regression.xlsx
✅ Results exported to 'tutorial_basic_regression.xlsx'
📁 Check your working directory for the Excel file!

🔍 Preview of the formatted table:
                      Variable      Model 1
                           age   192.515***
                                   (18.860)
                         const 20441.819***
                                 (1278.142)
                     education   597.978***
                                   (62.364)
                    experience   326.472***
                                   (25.722)
                                           
                  Observations         1000
                     R-squared        0.260
                   F-statistic       116.88
*** p<0.01, ** p<0.05, * p<0.1             


In [5]:
# Export to Word format  
outreg(result, 'tutorial_basic_regression.docx',
       title="Wage Regression Analysis", 
       ctitle="OLS Model",
       addnote="Standard errors in parentheses. Sample size: 1,000 observations.",
       replace=True)

print("✅ Results exported to 'tutorial_basic_regression.docx'")
print("📄 Open the Word document to see publication-ready formatting!")
print("\n💡 Key features of the output:")
print("   • Professional table formatting")
print("   • Significance stars (*** p<0.01, ** p<0.05, * p<0.1)")
print("   • Standard errors in parentheses")
print("   • Model statistics (R², F-stat, N)")
print("   • Custom titles and notes")

Results exported to tutorial_basic_regression.docx
✅ Results exported to 'tutorial_basic_regression.docx'
📄 Open the Word document to see publication-ready formatting!

💡 Key features of the output:
   • Professional table formatting
   • Significance stars (*** p<0.01, ** p<0.05, * p<0.1)
   • Standard errors in parentheses
   • Model statistics (R², F-stat, N)
   • Custom titles and notes


## 3. Multiple Model Comparison

One of the most powerful features of pyoutreg is the ability to compare multiple models side-by-side in a single table. This is essential for academic papers and research reports.

### 📋 What we'll cover:
- Fitting multiple regression models
- Using `outreg_compare()` for side-by-side comparison
- Adding model names and custom titles
- Interpreting the comparison table

In [6]:
# Fit multiple models with increasing complexity

# Model 1: Basic model (education only)
X1 = sm.add_constant(data[['education']])
model1 = sm.OLS(data['wage'], X1).fit()

# Model 2: Add experience
X2 = sm.add_constant(data[['education', 'experience']])
model2 = sm.OLS(data['wage'], X2).fit()

# Model 3: Add age and gender
X3 = sm.add_constant(data[['education', 'experience', 'age']])
model3 = sm.OLS(data['wage'], X3).fit()

# Model 4: Full model with gender dummy
data_with_gender = data.copy()
data_with_gender['male'] = (data_with_gender['gender'] == 'Male').astype(int)
X4 = sm.add_constant(data_with_gender[['education', 'experience', 'age', 'male']])
model4 = sm.OLS(data_with_gender['wage'], X4).fit()

# Display model comparison summary
models = [model1, model2, model3, model4]
model_names = ['Education Only', 'Add Experience', 'Add Age', 'Add Gender']

print("🎯 Model Comparison Summary:")
print("Model".ljust(20) + "R²".ljust(10) + "Adj. R²".ljust(12) + "N".ljust(8))
print("-" * 50)
for i, (model, name) in enumerate(zip(models, model_names)):
    print(f"{name:<20}{model.rsquared:.3f}    {model.rsquared_adj:.3f}     {int(model.nobs)}")

🎯 Model Comparison Summary:
Model               R²        Adj. R²     N       
--------------------------------------------------
Education Only      0.059    0.058     1000
Add Experience      0.183    0.181     1000
Add Age             0.260    0.258     1000
Add Gender          0.401    0.399     1000


In [7]:
# Export model comparison to Excel
outreg_compare(
    [model1, model2, model3, model4],
    'tutorial_model_comparison.xlsx',
    model_names=['Model 1', 'Model 2', 'Model 3', 'Model 4'],
    title='Wage Regression Models: Progressive Specification',
    replace=True
)

print("✅ Model comparison exported to 'tutorial_model_comparison.xlsx'")

# Preview the comparison table (now working with the fixed function!)
print("\n🔍 Preview of model comparison table:")
preview = outreg_compare(
    [model1, model2, model3, model4],
    filename=None,  # Return DataFrame instead of saving
    model_names=['Model 1', 'Model 2', 'Model 3', 'Model 4']
)
print(preview.to_string(index=False))

Comparison table exported to tutorial_model_comparison.xlsx
✅ Model comparison exported to 'tutorial_model_comparison.xlsx'

🔍 Preview of model comparison table:
                      Variable      Model 1      Model 2      Model 3      Model 4
                           age                             192.515***   194.849***
                                                             (18.860)     (16.980)
                         const 31267.511*** 27677.505*** 20441.819*** 18801.930***
                                 (1156.744)   (1117.252)   (1278.142)   (1155.675)
                     education   555.555***   563.789***   597.978***   541.228***
                                   (70.164)     (65.418)     (62.364)     (56.268)
                    experience                332.176***   326.472***   320.694***
                                                (27.014)     (25.722)     (23.160)
                          male                                         5117.938***
        

## 4. Summary Statistics

PyOutreg provides powerful functions for generating descriptive statistics tables, similar to Stata's `summarize` command.

### 📋 What we'll cover:
- Basic summary statistics
- Detailed statistics with percentiles
- Grouped statistics by categories
- Customizing variable selection

In [8]:
# Basic summary statistics
summary_stats(
    data, 
    'tutorial_summary_stats.xlsx',
    variables=['wage', 'education', 'experience', 'age', 'income'],
    title="Descriptive Statistics for Key Variables",
    replace=True
)

print("✅ Basic summary statistics exported to 'tutorial_summary_stats.xlsx'")

# Preview the summary statistics
print("\n🔍 Preview of summary statistics:")
summary_preview = summary_stats(
    data,
    filename=None,  # Return DataFrame
    variables=['wage', 'education', 'experience', 'age']
)
print(summary_preview.to_string(index=False))

Results exported to tutorial_summary_stats.xlsx
✅ Basic summary statistics exported to 'tutorial_summary_stats.xlsx'

🔍 Preview of summary statistics:
  Variable    N      Mean  Std Dev       Min       Max
      wage 1000 40275.871 6807.832 15000.000 61074.644
 education 1000    16.215    2.979     8.000    25.000
experience 1000    10.406    7.215     0.000    40.000
       age 1000    35.014    9.853    18.000    65.000


In [9]:
# Detailed summary statistics with percentiles
summary_stats(
    data,
    'tutorial_detailed_stats.xlsx',
    variables=['wage', 'education', 'experience'],
    detail=True,  # Include percentiles and skewness/kurtosis
    title="Detailed Descriptive Statistics",
    replace=True
)

print("✅ Detailed statistics exported to 'tutorial_detailed_stats.xlsx'")

# Grouped summary statistics
summary_stats(
    data,
    'tutorial_grouped_stats.xlsx',
    variables=['wage', 'education', 'experience'],
    by='gender',  # Group by gender
    title="Statistics by Gender",
    replace=True
)

print("✅ Grouped statistics exported to 'tutorial_grouped_stats.xlsx'")

# Preview grouped statistics
print("\n🔍 Preview of statistics by gender:")
grouped_preview = summary_stats(
    data,
    filename=None,
    variables=['wage', 'education'], 
    by='gender'
)
print(grouped_preview.to_string(index=False))

Results exported to tutorial_detailed_stats.xlsx
✅ Detailed statistics exported to 'tutorial_detailed_stats.xlsx'
Results exported to tutorial_grouped_stats.xlsx
✅ Grouped statistics exported to 'tutorial_grouped_stats.xlsx'

🔍 Preview of statistics by gender:
        Group  Variable   N         Mean     Std Dev          Min          Max
gender=Female      wage 504 37619.130906 6297.047178 15000.000000 56704.412321
gender=Female education 504    16.019050    2.927296     8.000000    23.768691
  gender=Male      wage 496 42975.462741 6225.194850 23728.780166 61074.643669
  gender=Male education 496    16.414236    3.020930     8.000000    25.000000


## 5. Cross-tabulation Tables

Cross-tabulation is essential for analyzing relationships between categorical variables. PyOutreg makes it easy to create professional cross-tabs with percentages.

### 📋 What we'll cover:
- Basic cross-tabulation between two categorical variables
- Interpreting the output with counts and percentages
- Formatting for publication

In [10]:
# Cross-tabulation: Gender by Region
cross_tab(
    data,
    'gender',       # Row variable
    'region',       # Column variable
    'tutorial_crosstab_gender_region.xlsx',
    title="Cross-tabulation: Gender by Region",
    replace=True
)

print("✅ Gender × Region cross-tab exported to 'tutorial_crosstab_gender_region.xlsx'")

# Preview the cross-tabulation
print("\n🔍 Preview of Gender × Region cross-tabulation:")
crosstab_preview = cross_tab(
    data,
    'gender',
    'region',
    filename=None  # Return DataFrame
)
print(crosstab_preview.to_string(index=False))

# Another example: Employment by Gender
cross_tab(
    data,
    'employment',
    'gender', 
    'tutorial_crosstab_employment_gender.xlsx',
    title="Employment Status by Gender",
    replace=True
)

print("\n✅ Employment × Gender cross-tab exported to 'tutorial_crosstab_employment_gender.xlsx'")

print("\n💡 Cross-tabulation features:")
print("   • Counts and percentages automatically calculated")
print("   • Row and column totals included")
print("   • Professional formatting for Excel export")
print("   • Easy to interpret percentages in parentheses")

Results exported to tutorial_crosstab_gender_region.xlsx
✅ Gender × Region cross-tab exported to 'tutorial_crosstab_gender_region.xlsx'

🔍 Preview of Gender × Region cross-tabulation:
gender\region        East       North       South        West Total
       Female 121 (50.8%) 126 (47.5%) 135 (50.0%) 122 (53.7%)   504
         Male 117 (49.2%) 139 (52.5%) 135 (50.0%) 105 (46.3%)   496
        Total         238         265         270         227  1000
Results exported to tutorial_crosstab_employment_gender.xlsx

✅ Employment × Gender cross-tab exported to 'tutorial_crosstab_employment_gender.xlsx'

💡 Cross-tabulation features:
   • Counts and percentages automatically calculated
   • Row and column totals included
   • Professional formatting for Excel export
   • Easy to interpret percentages in parentheses


## 6. Customization Options

PyOutreg offers extensive customization options to create exactly the tables you need for publication.

### 📋 What we'll cover:
- Decimal places control (dec, bdec, sdec)
- Variable selection (keep, drop)
- Custom titles and notes
- Additional statistics (addstat)
- Formatting options

In [19]:
# Advanced customization example
# First, let's fit a regression with more variables
X_full = sm.add_constant(data[['education', 'experience', 'age', 'income']])
result_full = sm.OLS(data['wage'], X_full).fit()

# Export with extensive customization
outreg(result_full, 'tutorial_customized.xlsx',
       replace=True,
       title="Wage Regression with Extensive Customization",
       ctitle="Full Model",
       
       # Decimal places control
       dec=2,          # Overall decimal places
       bdec=3,         # Coefficient decimal places  
       sdec=4,         # Standard error decimal places
       
       # Variable selection
       keep=['education', 'experience', 'age'],  # Only show these variables
       
       # Custom notes and statistics
       addnote="Robust standard errors. Income variable excluded from display but included in regression.",
       addstat={
           'Mean Wage': data['wage'].mean(),
           'Wage Std Dev': data['wage'].std(),
           'Sample Period': '2014-2023'
       },
       
       # Formatting
       font_size=11
)

print("✅ Customized table exported to 'tutorial_customized.xlsx'")

# Preview the customized output
print("\n🔍 Preview of customized table:")
customized_preview = outreg(result_full, 
                           filename=None,
                           ctitle="Full Model",
                           keep=['education', 'experience', 'age'],
                           bdec=3)
print(customized_preview.to_string(index=False))

Results exported to tutorial_customized.xlsx
✅ Customized table exported to 'tutorial_customized.xlsx'

🔍 Preview of customized table:
                      Variable Full Model
                           age 192.401***
                                 (18.870)
                     education 596.968***
                                 (62.442)
                    experience 326.764***
                                 (25.743)
                                         
                  Observations       1000
                     R-squared      0.261
                   F-statistic      87.63
*** p<0.01, ** p<0.05, * p<0.1           


## 7. Panel Data Models

PyOutreg supports linearmodels for panel data analysis, including fixed effects and random effects models.

### 📋 What we'll cover:
- Fixed effects regression
- Random effects regression  
- Comparing panel models
- Exporting panel results with pyoutreg

In [16]:
# Prepare panel data (we have 100 individuals over 10 years)
panel_data = data.copy()
panel_data = panel_data.set_index(['id', 'year'])

print(f"📊 Panel data structure:")
print(f"   • {panel_data.index.get_level_values('id').nunique()} individuals")
print(f"   • {panel_data.index.get_level_values('year').nunique()} time periods")
print(f"   • {len(panel_data):,} total observations")

# Fixed Effects Model
dependent = panel_data['wage']
exogenous = panel_data[['education', 'experience']]

# Fit fixed effects model
fe_model = lmp.PanelOLS(dependent, exogenous, entity_effects=True)
fe_result = fe_model.fit(cov_type='clustered', cluster_entity=True)

print(f"\n🎯 Fixed Effects Results:")
print(f"   • R-squared within: {fe_result.rsquared_within:.3f}")
print(f"   • R-squared overall: {fe_result.rsquared_overall:.3f}")
print(f"   • Number of entities: {fe_result.entity_info.total}")

# Export fixed effects results
outreg(fe_result, 'tutorial_panel_fe.xlsx',
       title="Panel Data Analysis: Fixed Effects Model",
       ctitle="Fixed Effects",
       addnote="Clustered standard errors by entity. Individual fixed effects included.",
       replace=True)

print("✅ Fixed effects results exported to 'tutorial_panel_fe.xlsx'")

📊 Panel data structure:
   • 100 individuals
   • 10 time periods
   • 1,000 total observations

🎯 Fixed Effects Results:
   • R-squared within: 0.182
   • R-squared overall: 0.529
   • Number of entities: 100.0
Results exported to tutorial_panel_fe.xlsx
✅ Fixed effects results exported to 'tutorial_panel_fe.xlsx'


In [18]:
# Random Effects Model
re_model = lmp.RandomEffects(dependent, exogenous)
re_result = re_model.fit()

print(f"🎯 Random Effects Results:")
print(f"   • R-squared overall: {re_result.rsquared_overall:.3f}")
# Note: theta is a property that might be a DataFrame, so let's handle it carefully
if hasattr(re_result, 'theta'):
    theta_val = re_result.theta
    if hasattr(theta_val, 'iloc'):  # It's a DataFrame or Series
        print(f"   • Theta: {float(theta_val.iloc[0]):.3f}")
    else:
        print(f"   • Theta: {theta_val:.3f}")

# Compare Fixed Effects vs Random Effects
outreg_compare(
    [fe_result, re_result],
    'tutorial_panel_comparison.xlsx',
    model_names=['Fixed Effects', 'Random Effects'],
    title='Panel Data Models Comparison',
    replace=True
)

print("✅ Panel models comparison exported to 'tutorial_panel_comparison.xlsx'")

# Preview the panel comparison
print("\n🔍 Preview of panel models comparison:")
panel_preview = outreg_compare(
    [fe_result, re_result],
    filename=None,
    model_names=['Fixed Effects', 'Random Effects']
)
print(panel_preview.to_string(index=False))

🎯 Random Effects Results:
   • R-squared overall: 0.963
   • Theta: 0.243
Comparison table exported to tutorial_panel_comparison.xlsx
✅ Panel models comparison exported to 'tutorial_panel_comparison.xlsx'

🔍 Preview of panel models comparison:
                      Variable Fixed Effects Random Effects
                     education    591.460***    2079.389***
                                    (66.302)       (27.647)
                    experience    320.815***     498.047***
                                    (30.645)       (32.980)
                                                           
                  Observations          1000           1000
                     R-squared         0.182          0.941
                   F-statistic                             
*** p<0.01, ** p<0.05, * p<0.1                             


## 8. Logistic Regression and Odds Ratios

PyOutreg supports logistic regression with the ability to export both coefficients and odds ratios, essential for binary outcome analysis.

### 📋 What we'll cover:
- Fitting logistic regression with statsmodels
- Exporting coefficient tables
- Exporting odds ratios using `eform=True`
- Interpreting logistic regression output

In [ ]:
# Prepare variables for logistic regression
y_binary = data['employed']  # Binary outcome (1=employed, 0=unemployed)
X_logit = sm.add_constant(data[['education', 'experience', 'age']])

# Fit logistic regression
logit_model = sm.Logit(y_binary, X_logit)
logit_result = logit_model.fit()

print("🎯 Logistic Regression Results:")
print(f"   • Pseudo R-squared: {logit_result.prsquared:.3f}")
print(f"   • Log-Likelihood: {logit_result.llf:.2f}")
print(f"   • LLR p-value: {logit_result.llr_pvalue:.4f}")
print(f"   • Number of observations: {int(logit_result.nobs):,}")

# Export coefficients
outreg(logit_result, 'tutorial_logit_coefficients.xlsx',
       title="Employment Probability: Logistic Regression",
       ctitle="Coefficients",
       addnote="Dependent variable: Employment status (1=employed, 0=unemployed)",
       replace=True)

print("✅ Logit coefficients exported to 'tutorial_logit_coefficients.xlsx'")

# Export odds ratios
outreg(logit_result, 'tutorial_logit_odds_ratios.xlsx',
       title="Employment Probability: Odds Ratios",
       ctitle="Odds Ratios",
       eform=True,  # Convert to odds ratios
       addnote="Odds ratios from logistic regression. Values > 1 indicate increased odds of employment.",
       replace=True)

print("✅ Logit odds ratios exported to 'tutorial_logit_odds_ratios.xlsx'")

# Preview both coefficient and odds ratio tables
print("\n🔍 Preview of logistic regression coefficients:")
coef_preview = outreg(logit_result, filename=None, ctitle="Coefficients")
print(coef_preview.to_string(index=False))

print("\n🔍 Preview of odds ratios:")
or_preview = outreg(logit_result, filename=None, ctitle="Odds Ratios", eform=True)
print(or_preview.to_string(index=False))

Optimization terminated successfully.
         Current function value: 0.639824
         Iterations 5
🎯 Logistic Regression Results:
   • Pseudo R-squared: 0.001
   • Log-Likelihood: -639.82
   • LLR p-value: 0.7790
   • Number of observations: 1,000
Results exported to tutorial_logit_coefficients.xlsx
✅ Logit coefficients exported to 'tutorial_logit_coefficients.xlsx'
Results exported to tutorial_logit_odds_ratios.xlsx
✅ Logit odds ratios exported to 'tutorial_logit_odds_ratios.xlsx'

🔍 Preview of logistic regression coefficients:
                      Variable Coefficients
                           age       -0.002
                                    (0.007)
                         const      1.109**
                                    (0.462)
                     education       -0.020
                                    (0.023)
                    experience       -0.005
                                    (0.009)
                                           
                  Obse

## 🎉 Congratulations! You've Completed the PyOutreg Tutorial

### 📋 What You've Learned:

1. **✅ Basic Regression Export** - OLS models to Excel and Word
2. **✅ Multiple Model Comparison** - Side-by-side model tables  
3. **✅ Summary Statistics** - Descriptive statistics with customization
4. **✅ Cross-tabulation** - Categorical variable analysis
5. **✅ Advanced Customization** - Titles, notes, decimal control, variable selection
6. **✅ Panel Data Models** - Fixed and random effects with linearmodels
7. **✅ Logistic Regression** - Coefficients and odds ratios

### 📁 Files Created in This Tutorial:

Check your working directory for these Excel and Word files:
- `tutorial_basic_regression.xlsx` & `.docx`
- `tutorial_model_comparison.xlsx`
- `tutorial_summary_stats.xlsx`
- `tutorial_detailed_stats.xlsx`
- `tutorial_grouped_stats.xlsx`
- `tutorial_crosstab_gender_region.xlsx`
- `tutorial_crosstab_employment_gender.xlsx`
- `tutorial_customized.xlsx`
- `tutorial_panel_fe.xlsx`
- `tutorial_panel_comparison.xlsx`
- `tutorial_logit_coefficients.xlsx`
- `tutorial_logit_odds_ratios.xlsx`

### 🚀 Next Steps:

1. **Try with your own data** - Replace our sample data with your research data
2. **Explore advanced features** - IV regression, more customization options
3. **Integrate into workflow** - Use pyoutreg in your research projects
4. **Share with colleagues** - Help others discover this powerful tool!

### 💡 Key Benefits of PyOutreg:

- **🎯 Saves Time** - No more manual table formatting
- **📊 Professional Output** - Publication-ready tables 
- **🔄 Reproducible** - Consistent formatting across projects
- **🛠️ Flexible** - Extensive customization options
- **📈 Comprehensive** - Supports multiple model types

### 📚 Additional Resources:

- **Documentation**: Check the `QUICKSTART.md` file
- **Examples**: Run `examples/basic_examples.py` for more examples  
- **Tests**: See `tests/test_pyoutreg.py` for validation
- **Source Code**: Explore the modular package structure

---

**Happy Analyzing! 📊✨**

*PyOutreg makes regression export in Python as easy as it is in Stata's outreg2!*

In [ ]:
# Final summary: Let's see what files we created!
import os
import glob

tutorial_files = glob.glob("tutorial_*.xlsx") + glob.glob("tutorial_*.docx")
tutorial_files.sort()

print("🎯 Tutorial Complete! Files created:")
print("=" * 50)

total_size = 0
for file in tutorial_files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        total_size += size
        print(f"✅ {file:<35} ({size:,} bytes)")

print("=" * 50)
print(f"📁 Total: {len(tutorial_files)} files ({total_size:,} bytes)")

print(f"\n🚀 PyOutreg Tutorial Summary:")
print(f"   • Regression exports: Excel & Word formats")
print(f"   • Multiple model comparisons")
print(f"   • Summary statistics & cross-tabulations") 
print(f"   • Panel data & logistic regression")
print(f"   • Professional publication-ready formatting")

print(f"\n💡 Ready to use PyOutreg in your research! 📊✨")

🎯 Tutorial Complete! Files created:
✅ tutorial_basic_regression.docx      (37,204 bytes)
✅ tutorial_basic_regression.xlsx      (5,363 bytes)
✅ tutorial_crosstab_employment_gender.xlsx (5,162 bytes)
✅ tutorial_crosstab_gender_region.xlsx (5,191 bytes)
✅ tutorial_customized.xlsx            (5,484 bytes)
✅ tutorial_detailed_stats.xlsx        (5,585 bytes)
✅ tutorial_grouped_stats.xlsx         (5,486 bytes)
✅ tutorial_logit_coefficients.xlsx    (5,394 bytes)
✅ tutorial_logit_odds_ratios.xlsx     (5,404 bytes)
✅ tutorial_model_comparison.xlsx      (5,731 bytes)
✅ tutorial_summary_stats.xlsx         (5,274 bytes)
📁 Total: 11 files (91,278 bytes)

🚀 PyOutreg Tutorial Summary:
   • Regression exports: Excel & Word formats
   • Multiple model comparisons
   • Summary statistics & cross-tabulations
   • Panel data & logistic regression
   • Professional publication-ready formatting

💡 Ready to use PyOutreg in your research! 📊✨
